In [1]:
from pyhive import hive
import pymysql
conn = hive.Connection(host='127.0.0.1', port=10000, username='hadoop')
cursor = conn.cursor()

In [2]:
def exec_sql(sql):
    cursor.execute(sql)
    for result in cursor.fetchall():
        print(result)

# 对缺失项进行预处理

# 实现hive和hbase映射

In [3]:
cursor.execute("DROP TABLE IF EXISTS film44")

In [4]:
mapping_from_hbase_to_hive_sql = '''CREATE EXTERNAL TABLE film44(id string, 
name string, 
male float, 
female float, 
under20 float, 
20to24 float, 
25to29 float, 
30to34 float, 
35to39 float, 
above40 float, 
degree_high float, 
degree_low float, 
city1 float, 
city2 float, 
city3 float, 
city4 float, 
occupation_whitecollar float, 
occupation_student float, 
occupation_others float, 
preference_action float, 
preference_comedy float, 
preference_romance float, 
occupation_science float, 
occupation_animate float, 
director string, 
actor1 string, 
actor2 string, 
actor3 string, 
actor4 string, 
t_romance int, 
t_comedy int, 
t_animate int, 
t_plot int, 
t_horror int, 
t_thriller int, 
t_science int, 
t_action int, 
t_suspense int, 
t_crime int, 
t_adventure int, 
t_war int, 
t_fantasy int, 
t_sports int, 
t_family int, 
t_costume int, 
t_sword int, 
t_cowboy int, 
t_history int, 
t_biography int, 
t_dance int, 
t_dark int, 
t_short int, 
t_documentary int, 
t_others int, 
c_Mainland int, 
c_America int, 
c_Korea int, 
c_Japan int, 
c_Hongkong int, 
c_Taiwan int, 
c_Thailand int, 
c_India int, 
c_France int, 
c_England int, 
c_Russia int, 
c_Italy int, 
c_Spain int, 
c_Germany int, 
c_Poland int, 
c_Australia int, 
c_Iran int, 
c_Others int, 
boxoffice float, 
mdate string, 
length int, 
score float, 
score_people float) 
STORED BY 'org.apache.hadoop.hive.hbase.HBaseStorageHandler' WITH SERDEPROPERTIES ("hbase.columns.mapping" = ":key,
f:n,
f:m,
f:fm,
f:un20,
f:20to24,
f:25to29,
f:30to34,
f:35to39,
f:ab40,
f:dg_h,
f:dg_l,
f:ct1,
f:ct2,
f:ct3,
f:ct4,
f:oc_w,
f:oc_s,
f:oc_o,
f:pf_ac,
f:pf_co,
f:pf_ro,
f:pf_sc,
f:pf_an,
f:dr,
f:a1,
f:a2,
f:a3,
f:a4,
f:t_aq,
f:t_xj,
f:t_dh,
f:t_jq,
f:t_kb,
f:t_js,
f:t_kh,
f:t_dz,
f:t_xy,
f:t_fz,
f:t_mx,
f:t_zz,
f:t_qh,
f:t_yd,
f:t_jt,
f:t_gz,
f:t_wx,
f:t_xb,
f:t_ls,
f:t_zj,
f:t_gw,
f:t_hs,
f:t_dp,
f:t_jl,
f:t_qt,
f:c_dl,
f:c_mg,
f:c_hg,
f:c_rb,
f:c_xg,
f:c_tw,
f:c_tg,
f:c_yd,
f:c_fg,
f:c_yg,
f:c_els,
f:c_ydl,
f:c_xby,
f:c_dg,
f:c_bl,
f:c_ad,
f:c_yl,
f:c_qt,
f:bof,
f:da,
f:len,
f:sc,
f:sc_p") TBLPROPERTIES("hbase.table.name" = "film22")
'''

In [5]:
cursor.execute(mapping_from_hbase_to_hive_sql)

In [6]:
exec_sql('select * from film44 limit 3')

('076b992c9d8585518bbdd5f2bd1a09a2', '谍影重重5', 0.516, 0.484, 0.078, 0.287, 0.278, 0.152, 0.101, 0.104, 0.305, 0.695, 0.149, 0.435, 0.195, 0.222, 0.36, 0.11, 0.52, 0.32, 0.12, 0.28, 0.17, 0.04, '保罗·格林格拉斯', '马特·达蒙', '艾丽西亚·维坎德', '朱丽娅·斯蒂尔斯', '汤米·李·琼斯', 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 446000000.0, '2016-08-23', 124, 8.3, 291000.0)
('0e953ed363842c143106e913a82f717d', '神战：权力之眼', 0.54, 0.46, 0.058, 0.305, 0.309, 0.155, 0.09, 0.083, 0.295, 0.705, 0.15, 0.453, 0.19, 0.207, 0.4, 0.09, 0.51, 0.32, 0.12, 0.28, 0.17, 0.04, '亚历克斯·普罗亚斯', '杰拉德·巴特勒', '尼古拉·科斯特-瓦尔道', '布兰顿·思怀兹', '查德维克·博斯曼', 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 235000000.0, '2016-03-11', 128, 8.5, 206000.0)
('1020349234a25c393c2f4ffe445c4b2c', '美国队长3', 0.517, 0.483, 0.085, 0.335, 0.279, 0.132, 0.083, 0.086, 0.303, 0.697, 0.147, 0.441, 0.192, 0.22, 0.35, 0

In [15]:
exec_sql('select count(*) from film44')

(59,)


# 依据需求文档进行数据提取

# table_1_district(地区分布）

In [67]:
cursor.execute("DROP TABLE IF EXISTS district")

In [68]:
create_table_district_sql = '''create table district(city1 float,city2 float,city3 float,city4 float)'''

In [69]:
cursor.execute(create_table_district_sql)

In [ ]:
del create_table_district_sql

In [38]:
district_insert_sql = 'insert into table district select city1,city2,city3,city4 from film44'

In [39]:
cursor.execute(district_insert_sql)

In [ ]:
del district_insert_sql

# table_2_relation_bet_gen_and_bo（性别和票房的关系）

In [60]:
cursor.execute("DROP TABLE IF EXISTS relation_bet_gen_and_bo")

In [61]:
create_table_relation_bet_gen_and_bo_sql = '''create table relation_bet_gen_and_bo(male float,female float,boxoffice float)'''

In [62]:
cursor.execute(create_table_relation_bet_gen_and_bo_sql)

In [ ]:
del create_table_relation_bet_gen_and_bo_sql

In [65]:
relation_bet_gen_and_bo_insert_sql = 'insert into table relation_bet_gen_and_bo select male,female,boxoffice from film44'

In [66]:
cursor.execute(relation_bet_gen_and_bo_insert_sql)

In [ ]:
del relation_bet_gen_and_bo_insert_sql

# table_3_female_per_by_hyear(每半年统计一次）

In [2]:
cursor.execute("DROP TABLE IF EXISTS female_per_by_hyear")

In [3]:
create_female_per_by_hyear_sql = '''create table female_per_by_hyear(mdate string,female_per float)'''

In [4]:
cursor.execute(create_female_per_by_hyear_sql)

In [ ]:
del create_female_per_by_hyear_sql

In [5]:
sixt_fh_sql ='''insert into table female_per_by_hyear SELECT '2016fh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <='2016-05-31' AND t1.mdate>='2016-01-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2016-12-31' AND t3.mdate>='2016-01-01') t4'''

In [6]:
cursor.execute(sixt_fh_sql)

In [ ]:
del sixt_fh_sql

In [7]:
sixt_sh_sql = '''insert into table female_per_by_hyear SELECT '2016sh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <'2016-12-31' AND t1.mdate>'2016-06-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2016-12-31' AND t3.mdate>='2016-01-01') t4'''

In [8]:
cursor.execute(sixt_sh_sql)

In [ ]:
del sixt_sh_sql

In [24]:
sevente_fh_sql = '''insert into table female_per_by_hyear SELECT '2017fh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <='2017-05-31' AND t1.mdate>='2017-01-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2017-12-31' AND t3.mdate>='2017-01-01') t4'''

In [25]:
cursor.execute(sevente_fh_sql)

In [ ]:
del sevente_fh_sql

In [26]:
sevent_sh_sql= '''insert into table female_per_by_hyear SELECT '2017sh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <='2017-12-31' AND t1.mdate>='2017-06-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2017-12-31' AND t3.mdate>='2017-01-01') t4'''

In [27]:
cursor.execute(sevent_sh_sql)

In [ ]:
del sevent_sh_sql

In [28]:
eighte_fh_sql = '''insert into table female_per_by_hyear SELECT '2018fh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <='2018-05-31' AND t1.mdate>='2018-01-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2018-12-31' AND t3.mdate>='2018-01-01') t4'''

In [29]:
cursor.execute(eighte_fh_sql)

In [ ]:
del eighte_fh_sql

In [30]:
eighte_sh_sql = '''insert into table female_per_by_hyear SELECT '2018sh',(t2.v / t4.s) * 100.0 FROM (SELECT SUM(t1.female) AS v FROM film44 t1 WHERE t1.mdate <='2018-12-31' AND t1.mdate>='2018-06-01') t2 JOIN (SELECT sum(t3.female) as s FROM film44 t3 WHERE t3.mdate <='2018-12-31' AND t3.mdate>='2018-01-01') t4'''

In [31]:
cursor.execute(eighte_sh_sql)

In [ ]:
del eighte_sh_sql

# table_4_relation_bet_gen_and_class(性别和电影种类的关系)

In [4]:
cursor.execute("DROP TABLE IF EXISTS relation_bet_gen_and_class")

In [5]:
create_relation_bet_gen_and_class_sql = '''create table relation_bet_gen_and_class(class string,male float,female float)'''

In [6]:
cursor.execute(create_relation_bet_gen_and_class_sql)

In [ ]:
del create_relation_bet_gen_and_class_sql

In [2]:
t_romance_sql = '''insert into table relation_bet_gen_and_class SELECT 't_romance',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_romance =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_romance =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_romance =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_romance =1) t8'''

In [3]:
cursor.execute(t_romance_sql)

In [ ]:
del t_romance_sql

In [4]:
t_comedy_sql='''insert into table relation_bet_gen_and_class SELECT 't_comedy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_comedy =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_comedy =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_comedy =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_comedy =1) t8'''

In [5]:
cursor.execute(t_comedy_sql)

In [ ]:
del t_comedy_sql

In [6]:
t_animate_sql = '''insert into table relation_bet_gen_and_class SELECT 't_animate',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_animate =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_animate =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_animate =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_animate =1) t8'''

In [7]:
cursor.execute(t_animate_sql)

In [ ]:
del t_animate_sql

In [8]:
t_plot_sql = '''insert into table relation_bet_gen_and_class SELECT 't_plot',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_plot =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_plot =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_plot =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_plot =1) t8'''

In [9]:
cursor.execute(t_plot_sql)

In [ ]:
del t_plot_sql

In [ ]:
t_horrors_sql = '''insert into table relation_bet_gen_and_class SELECT 't_horror',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_horror =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_horror =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_horror =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_horror =1) t8'''

In [ ]:
cursor.execute(t_horrors_sql)

In [ ]:
del t_horrors_sql

In [ ]:
t_thriller_sql = '''insert into table relation_bet_gen_and_class SELECT 't_thriller',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_thriller =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_thriller =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_thriller =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_thriller =1) t8'''

In [ ]:
cursor.execute(t_thriller_sql)

In [ ]:
del t_thriller_sql

In [ ]:
t_science_sql = '''insert into table relation_bet_gen_and_class SELECT 't_science',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_science =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_science =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_science =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_science =1) t8'''

In [ ]:
cursor.execute(t_science_sql)

In [ ]:
del t_science_sql

In [ ]:
动作

In [ ]:
t_action_sql = '''insert into table relation_bet_gen_and_class SELECT 't_action',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_action =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_action =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_action =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_action =1) t8'''

In [ ]:
cursor.execute(t_action_sql)

In [ ]:
del t_action_sql

In [ ]:
t_suspense_sql = '''insert into table relation_bet_gen_and_class SELECT 't_suspense',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_suspense =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_suspense =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_suspense =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_suspense =1) t8'''

In [ ]:
cursor.execute(t_suspense_sql)

In [ ]:
del t_suspense_sql

In [ ]:
t_crime_sql = '''insert into table relation_bet_gen_and_class SELECT 't_crime',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_crime =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_crime =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_crime =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_crime =1) t8'''

In [ ]:
cursor.execute(t_crime_sql)

In [ ]:
del t_crime_sql

In [ ]:
t_adventure_sql = '''insert into table relation_bet_gen_and_class SELECT 't_adventure',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_adventure =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_adventure =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_adventure =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_adventure =1) t8'''

In [ ]:
cursor.execute(t_adventure_sql)

In [ ]:
del t_adventure_sql

In [ ]:
t_war_sql = '''insert into table relation_bet_gen_and_class SELECT 't_war',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_war =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_war =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_war =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_war =1) t8'''

In [ ]:
cursor.execute(t_war_sql)

In [ ]:
del t_war_sql

In [ ]:
t_fantasy_sql = '''insert into table relation_bet_gen_and_class SELECT 't_fantasy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_fantasy =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_fantasy =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_fantasy =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_fantasy =1) t8'''

In [ ]:
cursor.execute(t_fantasy_sql)

In [ ]:
del t_fantasy_sql

In [ ]:
t_sports_sql = '''insert into table relation_bet_gen_and_class SELECT 't_sports',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_sports =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_sports =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_sports =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_sports =1) t8'''

In [ ]:
cursor.execute(t_sports_sql)

In [ ]:
del t_sports_sql

In [ ]:
t_family_sql = '''insert into table relation_bet_gen_and_class SELECT 't_family',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_family =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_family =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_family =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_family =1) t8'''

In [ ]:
cursor.execute(t_family_sql)

In [ ]:
del t_family_sql

In [ ]:
古装

In [ ]:
t_costume_sql = '''insert into table relation_bet_gen_and_class SELECT 't_costume',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_costume =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_costume =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_costume =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_costume =1) t8'''

In [ ]:
cursor.execute(t_costume_sql)

In [ ]:
del t_costume_sql

In [ ]:
t_sword_sql = '''insert into table relation_bet_gen_and_class SELECT 't_sword',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_sword =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_sword =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_sword =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_sword =1) t8'''

In [ ]:
cursor.execute(t_sword_sql)

In [ ]:
del t_sword_sql

In [ ]:
t_cowboy_sql = '''insert into table relation_bet_gen_and_class SELECT 't_cowboy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_cowboy =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_cowboy =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_cowboy =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_cowboy =1) t8'''

In [ ]:
cursor.execute(t_cowboy_sql)

In [ ]:
del t_cowboy_sql

In [ ]:
t_history_sql = '''insert into table relation_bet_gen_and_class SELECT 't_history',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_history =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_history =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_history =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_history =1) t8'''

In [ ]:
cursor.execute(t_history_sql)

In [ ]:
del t_history_sql

In [ ]:
t_biography_sql = '''insert into table relation_bet_gen_and_class SELECT 't_biography',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_biography =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_biography =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_biography =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_biography =1) t8'''

In [ ]:
cursor.execute(t_biography_sql)

In [ ]:
del t_biography_sql

In [ ]:
t_dance_sql = '''insert into table relation_bet_gen_and_class SELECT 't_dance',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_dance =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_dance =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_dance =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_dance =1) t8'''

In [ ]:
cursor.execute(t_dance_sql)

In [ ]:
del t_dance_sql

In [ ]:
t_dark_sql = '''insert into table relation_bet_gen_and_class SELECT 't_dark',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_dark =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_dark =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_dark =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_dark =1) t8'''

In [ ]:
cursor.execute(t_dark_sql)

In [ ]:
del t_dark_sql

In [ ]:
t_short_sql = '''insert into table relation_bet_gen_and_class SELECT 't_short',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_short =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_short=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_short=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_short=1) t8'''

In [ ]:
cursor.execute(t_short_sql)

In [ ]:
del t_short_sql

In [ ]:
t_documentary_sql = '''insert into table relation_bet_gen_and_class SELECT 't_documentary',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_documentary =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_documentary=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_documentary=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_documentary=1) t8'''

In [ ]:
cursor.execute(t_documentary_sql)

In [ ]:
del t_documentary_sql

In [ ]:
t_others_sql = '''insert into table relation_bet_gen_and_class SELECT 't_others',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.t_others =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.t_others=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.t_others=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.t_others=1) t8'''

In [ ]:
cursor.execute(t_others_sql)

In [ ]:
del t_others_sql

In [ ]:
c_Mainland_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Mainland',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Mainland =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Mainland=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Mainland=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Mainland=1) t8'''

In [ ]:
cursor.execute(c_Mainland_sql)

In [ ]:
del c_Mainland_sql

In [ ]:
c_America_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_America',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_America =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_America=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_America=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_America=1) t8'''

In [ ]:
cursor.execute(c_America_sql)

In [ ]:
del c_America_sql

In [ ]:
c_Korea_sql ='''insert into table relation_bet_gen_and_class SELECT 'c_Korea',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Korea =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Korea=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Korea=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Korea=1) t8'''

In [ ]:
cursor.execute(c_Korea_sql)

In [ ]:
del c_Korea_sql

In [ ]:
c_Japan_sql ='''insert into table relation_bet_gen_and_class SELECT 'c_Japan',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Japan =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Japan=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Japan=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Japan=1) t8'''

In [ ]:
cursor.execute(c_Japan_sql)

In [ ]:
del c_Japan_sql

In [ ]:
c_Hongkong_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Hongkong',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Hongkong =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Hongkong=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Hongkong=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Hongkong=1) t8'''

In [ ]:
cursor.execute(c_Hongkong_sql)

In [ ]:
del c_Hongkong_sql

In [ ]:
c_Taiwan_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Taiwan',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Taiwan =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Taiwan=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Taiwan=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Taiwan=1) t8'''

In [ ]:
cursor.execute(c_Taiwan_sql)

In [ ]:
del c_Taiwan_sql

In [ ]:
c_Thailand_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Thailand',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Thailand =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Thailand=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Thailand=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Thailand=1) t8'''

In [ ]:
cursor.execute(c_Thailand_sql)

In [ ]:
del c_Thailand_sql

In [ ]:
c_India_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_India',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_India =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_India=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_India=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_India=1) t8'''

In [ ]:
cursor.execute(c_India_sql)

In [ ]:
del c_India_sql

In [ ]:
c_France_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_France',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_France =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_France=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_France=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_France=1) t8'''

In [ ]:
cursor.execute(c_France_sql)

In [ ]:
del c_France_sql

In [ ]:
c_England_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_England',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_England =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_England=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_England=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_England=1) t8'''

In [ ]:
cursor.execute(c_England_sql)

In [ ]:
del c_England_sql

In [ ]:
c_Russia_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Russia',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Russia =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Russia=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Russia=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Russia=1) t8'''

In [ ]:
cursor.execute(c_Russia_sql)

In [ ]:
del c_Russia_sql

In [ ]:
c_Italy_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Italy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Italy =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Italy =1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Italy =1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Italy =1) t8'''

In [ ]:
cursor.execute(c_Italy_sql)

In [ ]:
del c_Italy_sql

In [ ]:
c_Spain_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Spain',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Spain =1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Spain=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Spain=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Spain =1) t8'''

In [ ]:
cursor.execute(c_Spain_sql)

In [ ]:
del c_Spain_sql

In [ ]:
c_Germany_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Germany',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Germany=1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Germany=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Germany=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Germany=1) t8'''

In [ ]:
cursor.execute(c_Germany_sql)

In [ ]:
del c_Germany_sql

In [ ]:
c_Poland_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Poland',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Poland=1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Poland=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Poland=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Poland=1) t8'''

In [ ]:
cursor.execute(c_Poland_sql)

In [ ]:
del c_Poland_sql

In [ ]:
c_Australia_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Australia',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Australia=1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Australia=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Australia=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Australia=1) t8'''

In [ ]:
cursor.execute(c_Australia_sql)

In [ ]:
del c_Australia_sql

In [ ]:
c_Iran_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Iran',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Iran=1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Iran=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Iran=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Iran=1) t8'''

In [ ]:
cursor.execute(c_Iran_sql)

In [ ]:
del c_Iran_sql

In [10]:
c_Others_sql = '''insert into table relation_bet_gen_and_class SELECT 'c_Others',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0 FROM (SELECT SUM(t1.male) AS v FROM film44 t1 WHERE t1.c_Others=1) t2 JOIN (SELECT count(t3.male) as s FROM film44 t3 WHERE t3.c_Others=1) t4 JOIN (SELECT SUM(t5.female) AS v FROM film44 t5 WHERE t5.c_Others=1) t6 JOIN (SELECT count(t7.female) as s FROM film44 t7 WHERE t7.c_Others=1) t8'''

In [ ]:
cursor.execute(c_Others_sql)

In [ ]:
del c_Others_sql

# table_5_relation_bet_age_and_class(年龄和类别的关系）

In [7]:
cursor.execute("DROP TABLE IF EXISTS relation_bet_age_and_class")

In [8]:
create_relation_bet_age_and_class_sql = '''create table relation_bet_age_and_class(class string,under20 float,20to24 float,25to29 float,30to34 float,35to39 float,above40 float)'''

In [9]:
cursor.execute(create_relation_bet_age_and_class_sql)

In [ ]:
del create_relation_bet_age_and_class_sql

In [ ]:
t_romance_sql = '''insert into table relation_bet_age_and_class SELECT 't_romance',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_romance=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_romance=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_romance=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_romance=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_romance=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_romance=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_romance=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_romance=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_romance=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_romance=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_romance=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_romance=1) t24'''

In [ ]:
cursor.execute(t_romance_sql)

In [ ]:
del t_romance_sql

In [ ]:
t_comedy_sql = '''insert into table relation_bet_age_and_class SELECT 't_comedy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_comedy=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_comedy=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_comedy=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_comedy=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_comedy=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_comedy=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_comedy=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_comedy=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_comedy=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_comedy=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_comedy=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_comedy=1) t24'''

In [ ]:
cursor.execute(t_comedy_sql)

In [ ]:
del t_comedy_sql

In [ ]:
t_animate_sql = '''insert into table relation_bet_age_and_class SELECT 't_animate',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_animate=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_animate=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_animate=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_animate=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_animate=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_animate=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_animate=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_animate=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_animate=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_animate=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_animate=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_animate=1) t24'''

In [ ]:
cursor.execute(t_animate_sql)

In [ ]:
del t_animate_sql

In [ ]:
t_plot_sql = '''insert into table relation_bet_age_and_class SELECT 't_plot',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_plot=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_plot=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_plot=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_plot=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_plot=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_plot=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_plot=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_plot=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_plot=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_plot=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_plot=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_plot=1) t24'''

In [ ]:
cursor.execute(t_plot_sql)

In [ ]:
del t_plot_sql

In [ ]:
t_horror_sql = '''insert into table relation_bet_age_and_class SELECT 't_horror',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_horror=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_horror=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_horror=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_horror=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_horror=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_horror=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_horror=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_horror=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_horror=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_horror=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_horror=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_horror=1) t24'''

In [ ]:
cursor.execute(t_horror_sql)

In [ ]:
del t_horror_sql

In [ ]:
t_thriller_sql = '''insert into table relation_bet_age_and_class SELECT 't_thriller',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_thriller=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_thriller=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_thriller=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_thriller=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_thriller=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_thriller=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_thriller=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_thriller=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_thriller=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_thriller=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_thriller=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_thriller=1) t24'''

In [ ]:
cursor.execute(t_thriller_sql)

In [ ]:
del t_thriller_sql

In [ ]:
t_science_sql = '''insert into table relation_bet_age_and_class SELECT 't_science',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_science=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_science=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_science=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_science=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_science=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_science=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_science=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_science=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_science=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_science=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_science=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_science=1) t24'''

In [ ]:
cursor.execute(t_science_sql)

In [ ]:
del t_science_sql

In [ ]:
t_action_sql = '''insert into table relation_bet_age_and_class SELECT 't_action',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_action=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_action=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_action=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_action=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_action=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_action=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_action=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_action=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_action=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_action=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_action=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_action=1) t24'''

In [ ]:
cursor.execute(t_science_sql)

In [ ]:
del t_action_sql

In [ ]:
t_suspense_sql = '''insert into table relation_bet_age_and_class SELECT 't_suspense',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_suspense=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_suspense=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_suspense=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_suspense=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_suspense=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_suspense=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_suspense=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_suspense=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_suspense=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_suspense=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_suspense=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_suspense=1) t24'''

In [ ]:
cursor.execute(t_suspense_sql)

In [ ]:
del t_suspense_sql

In [ ]:
t_crime_sql = '''insert into table relation_bet_age_and_class SELECT 't_crime',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_crime=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_crime=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_crime=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_crime=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_crime=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_crime=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_crime=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_crime=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_crime=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_crime=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_crime=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_crime=1) t24'''

In [ ]:
cursor.execute(t_crime_sql)

In [ ]:
del t_crime_sql

In [ ]:
t_adventure_sql = '''insert into table relation_bet_age_and_class SELECT 't_adventure',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_adventure=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_adventure=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_adventure=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_adventure=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_adventure=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_adventure=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_adventure=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_adventure=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_adventure=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_adventure=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_adventure=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_adventure=1) t24'''

In [ ]:
cursor.execute(t_adventure_sql)

In [ ]:
del t_adventure_sql

In [ ]:
t_war_sql = '''insert into table relation_bet_age_and_class SELECT 't_war',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_war=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_war=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_war=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_war=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_war=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_war=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_war=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_war=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_war=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_war=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_war=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_war=1) t24'''

In [ ]:
cursor.execute(t_war_sql)

In [ ]:
del t_war_sql

In [ ]:
t_fantasy_sql = '''insert into table relation_bet_age_and_class SELECT 't_fantasy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_fantasy=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_fantasy=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_fantasy=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_fantasy=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_fantasy=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_fantasy=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_fantasy=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_fantasy=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_fantasy=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_fantasy=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_fantasy=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_fantasy=1) t24'''

In [ ]:
cursor.execute(t_fantasy_sql)

In [ ]:
del t_fantasy_sql

In [10]:
t_sports_sql = '''insert into table relation_bet_age_and_class SELECT 't_sports',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_sports=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_sports=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_sports=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_sports=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_sports=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_sports=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_sports=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_sports=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_sports=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_sports=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_sports=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_sports=1) t24'''

In [11]:
cursor.execute(t_sports_sql)

In [ ]:
del t_sports_sql

In [ ]:
t_family_sql = '''insert into table relation_bet_age_and_class SELECT 't_family',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_family=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_family=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_family=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_family=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_family=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_family=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_family=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_family=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_family=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_family=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_family=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_family=1) t24'''

In [ ]:
cursor.execute(t_family_sql)

In [ ]:
del t_family_sql

In [ ]:
t_costume_sql = '''insert into table relation_bet_age_and_class SELECT 't_costume',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_costume=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_costume=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_costume=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_costume=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_costume=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_costume=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_costume=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_costume=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_costume=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_costume=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_costume=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_costume=1) t24'''

In [ ]:
cursor.execute(t_costume_sql)

In [ ]:
del t_costume_sql

In [ ]:
t_sword_sql = '''insert into table relation_bet_age_and_class SELECT 't_sword',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_sword=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_sword=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_sword=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_sword=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_sword=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_sword=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_sword=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_sword=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_sword=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_sword=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_sword=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_sword=1) t24'''

In [ ]:
cursor.execute(t_sword_sql)

In [ ]:
del t_sword_sql

In [ ]:
t_cowboy_sql = '''insert into table relation_bet_age_and_class SELECT 't_cowboy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_cowboy=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_cowboy=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_cowboy=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_cowboy=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_cowboy=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_cowboy=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_cowboy=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_cowboy=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_cowboy=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_cowboy=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_cowboy=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_cowboy=1) t24'''

In [ ]:
cursor.execute(t_cowboy_sql)

In [ ]:
del t_cowboy_sql

In [ ]:
t_history_sql = '''insert into table relation_bet_age_and_class SELECT 't_history',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_history=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_history=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_history=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_history=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_history=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_history=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_history=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_history=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_history=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_history=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_history=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_history=1) t24'''

In [ ]:
cursor.execute(t_history_sql)

In [ ]:
del t_history_sql

In [ ]:
t_biography_sql = '''insert into table relation_bet_age_and_class SELECT 't_biography',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_biography=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_biography=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_biography=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_biography=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_biography=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_biography=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_biography=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_biography=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_biography=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_biography=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_biography=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_biography=1) t24'''

In [ ]:
cursor.execute(t_biography_sql)

In [ ]:
del t_biography_sql

In [ ]:
t_dance_sql = '''insert into table relation_bet_age_and_class SELECT 't_dance',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_dance=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_dance=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_dance=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_dance=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_dance=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_dance=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_dance=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_dance=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_dance=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_dance=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_dance=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_dance=1) t24'''

In [ ]:
cursor.execute(t_dance_sql)

In [ ]:
del t_dance_sql

In [ ]:
t_dark_sql = '''insert into table relation_bet_age_and_class SELECT 't_dark',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_dark=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_dark=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_dark=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_dark=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_dark=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_dark=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_dark=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_dark=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_dark=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_dark=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_dark=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_dark=1) t24'''

In [ ]:
cursor.execute(t_dark_sql)

In [ ]:
del t_dark_sql

In [ ]:
t_short_sql = '''insert into table relation_bet_age_and_class SELECT 't_short',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_short=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_short=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_short=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_short=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_short=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_short=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_short=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_short=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_short=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_short=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_short=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_short=1) t24'''

In [ ]:
cursor.execute(t_short_sql)

In [ ]:
del t_short_sql

In [ ]:
t_documentary_sql = '''insert into table relation_bet_age_and_class SELECT 't_documentary',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_documentary=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_documentary=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_documentary=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_documentary=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_documentary=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_documentary=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_documentary=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_documentary=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_documentary=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_documentary=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_documentary=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_documentary=1) t24'''

In [ ]:
cursor.execute(t_documentary_sql)

In [ ]:
del t_documentary_sql

In [ ]:
t_others_sql = '''insert into table relation_bet_age_and_class SELECT 't_others',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.t_others=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.t_others=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.t_others=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.t_others=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.t_others=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.t_others=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.t_others=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.t_others=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.t_others=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.t_others=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.t_others=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.t_others=1) t24'''

In [ ]:
cursor.execute(t_others_sql)

In [ ]:
del t_others_sql

In [ ]:
c_Mainland_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Mainland',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Mainland=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Mainland=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Mainland=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Mainland=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Mainland=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Mainland=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Mainland=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Mainland=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Mainland=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Mainland=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Mainland=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Mainland=1) t24'''

In [ ]:
cursor.execute(c_Mainland_sql)

In [ ]:
del c_Mainland_sql

In [ ]:
c_America_sql = '''insert into table relation_bet_age_and_class SELECT 'c_America',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_America=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_America=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_America=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_America=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_America=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_America=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_America=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_America=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_America=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_America=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_America=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_America=1) t24'''

In [ ]:
cursor.execute(c_America_sql)

In [ ]:
del c_America_sql

In [ ]:
c_Korea_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Korea',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Korea=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Korea=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Korea=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Korea=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Korea=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Korea=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Korea=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Korea=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Korea=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Korea=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Korea=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Korea=1) t24'''

In [ ]:
cursor.execute(c_America_sql)

In [ ]:
del c_Korea_sql

In [ ]:
c_Japan_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Japan',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Japan=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Japan=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Japan=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Japan=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Japan=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Japan=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Japan=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Japan=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Japan=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Japan=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Japan=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Japan=1) t24'''

In [ ]:
cursor.execute(c_Japan_sql)

In [ ]:
del c_Japan_sql

In [ ]:
c_Hongkong_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Hongkong',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Hongkong=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Hongkong=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Hongkong=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Hongkong=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Hongkong=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Hongkong=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Hongkong=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Hongkong=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Hongkong=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Hongkong=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Hongkong=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Hongkong=1) t24'''

In [ ]:
cursor.execute(c_Hongkong_sql)

In [ ]:
del c_Hongkong_sql

In [ ]:
c_Taiwan_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Taiwan',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Taiwan=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Taiwan=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Taiwan=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Taiwan=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Taiwan=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Taiwan=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Taiwan=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Taiwan=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Taiwan=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Taiwan=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Taiwan=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Taiwan=1) t24'''

In [ ]:
cursor.execute(c_Taiwan_sql)

In [ ]:
del c_Taiwan_sql

In [ ]:
c_Thailand_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Thailand',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Thailand=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Thailand=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Thailand=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Thailand=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Thailand=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Thailand=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Thailand=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Thailand=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Thailand=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Thailand=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Thailand=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Thailand=1) t24'''

In [ ]:
cursor.execute(c_Thailand_sql)

In [ ]:
del c_Thailand_sql

In [ ]:
c_India_sql = '''insert into table relation_bet_age_and_class SELECT 'c_India',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_India=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_India=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_India=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_India=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_India=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_India=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_India=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_India=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_India=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_India=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_India=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_India=1) t24'''

In [ ]:
cursor.execute(c_India_sql)

In [ ]:
del c_India_sql

In [ ]:
c_France_sql = '''insert into table relation_bet_age_and_class SELECT 'c_France',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_France=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_France=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_France=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_France=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_France=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_France=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_France=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_France=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_France=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_France=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_France=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_France=1) t24'''

In [ ]:
cursor.execute(c_France_sql)

In [ ]:
del c_France_sql

In [ ]:
c_England_sql = '''insert into table relation_bet_age_and_class SELECT 'c_England',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_England=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_England=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_England=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_England=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_England=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_England=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_England=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_England=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_England=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_England=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_England=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_England=1) t24'''

In [ ]:
cursor.execute(c_England_sql)

In [ ]:
del c_England_sql

In [ ]:
c_Russia_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Russia',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Russia=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Russia=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Russia=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Russia=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Russia=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Russia=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Russia=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Russia=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Russia=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Russia=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Russia=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Russia=1) t24'''

In [ ]:
cursor.execute(c_Russia_sql)

In [ ]:
del c_Russia_sql

In [ ]:
c_Italy_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Italy',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Italy=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Italy=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Italy=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Italy=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Italy=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Italy=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Italy=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Italy=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Italy=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Italy=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Italy=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Italy=1) t24'''

In [ ]:
cursor.execute(c_Italy_sql)

In [ ]:
del c_Italy_sql

In [ ]:
c_Spain_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Spain',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Spain=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Spain=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Spain=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Spain=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Spain=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Spain=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Spain=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Spain=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Spain=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Spain=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Spain=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Spain=1) t24'''

In [ ]:
cursor.execute(c_Spain_sql)

In [ ]:
del c_Spain_sql

In [ ]:
c_Germany_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Germany',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Germany=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Germany=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Germany=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Germany=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Germany=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Germany=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Germany=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Germany=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Germany=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Germany=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Germany=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Germany=1) t24'''

In [ ]:
cursor.execute(c_Germany_sql)

In [ ]:
del c_Germany_sql

In [ ]:
波兰

In [ ]:
c_Poland_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Poland',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Poland=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Poland=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Poland=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Poland=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Poland=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Poland=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Poland=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Poland=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Poland=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Poland=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Poland=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Poland=1) t24'''

In [ ]:
cursor.execute(c_Poland_sql)

In [ ]:
del c_Poland_sql

In [ ]:
c_Australia_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Australia',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Australia=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Australia=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Australia=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Australia=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Australia=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Australia=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Australia=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Australia=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Australia=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Australia=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Australia=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Australia=1) t24'''

In [ ]:
cursor.execute(c_Australia_sql)

In [ ]:
del c_Australia_sql

In [ ]:
c_Iran_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Iran',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Iran=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Iran=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Iran=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Iran=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Iran=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Iran=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Iran=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Iran=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Iran=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Iran=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Iran=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Iran=1) t24'''

In [ ]:
cursor.execute(c_Iran_sql)

In [ ]:
del c_Iran_sql

In [ ]:
其他

In [ ]:
c_Others_sql = '''insert into table relation_bet_age_and_class SELECT 'c_Others',(t2.v / t4.s) * 100.0,(t6.v / t8.s) * 100.0,(t10.v / t12.s) * 100.0,(t14.v / t16.s) * 100.0,(t18.v / t20.s) * 100.0,(t22.v / t24.s) * 100.0               FROM (SELECT SUM(t1.under20) AS v FROM film44 t1 WHERE t1.c_Others=1) t2 JOIN (SELECT count(t3.under20) as s FROM film44 t3 WHERE t3.c_Others=1) t4 JOIN (SELECT SUM(t5.20to24) AS v FROM film44 t5 WHERE t5.c_Others=1) t6 JOIN (SELECT count(t7.20to24) as s FROM film44 t7 WHERE t7.c_Others=1) t8 JOIN (SELECT SUM(t9.25to29) AS v FROM film44 t9 WHERE t9.c_Others=1) t10 JOIN (SELECT count(t11.25to29) as s FROM film44 t11 WHERE t11.c_Others=1) t12 JOIN (SELECT SUM(t13.30to34) AS v FROM film44 t13 WHERE t13.c_Others=1) t14 JOIN (SELECT count(t15.30to34) as s FROM film44 t15 WHERE t15.c_Others=1) t16 JOIN (SELECT SUM(t17.35to39) AS v FROM film44 t17 WHERE t17.c_Others=1) t18 JOIN (SELECT count(t19.35to39) as s FROM film44 t19 WHERE t19.c_Others=1) t20 JOIN (SELECT SUM(t21.above40) AS v FROM film44 t21 WHERE t21.c_Others=1) t22 JOIN (SELECT count(t23.above40) as s FROM film44 t23 WHERE t23.c_Others=1) t24'''

In [ ]:
cursor.execute(c_Others_sql)

In [ ]:
del c_Others_sql

# table_6_

# 写进数据库

# 用pandas查看一下

In [21]:
#打开数据库连接
db= pymysql.connect(host="127.0.0.1",user="root",
     password="123456",db="comex",port=3306)

In [22]:
import pandas as pd

In [23]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', None)

In [24]:
sql = 'select * from personas'
pd = pd.read_sql(sql,db)

In [25]:
pd.head(5)

,id,name,male,female,under20,20to24,25to29,30to34,35to39,above40,degree_high,degree_low,city1,city2,city3,city4,occupation_whitecollar,occupation_student,occupation_others,preference_action,preference_comedy,preference_romance,occupation_science,occupation_animate
0,c2ea3dd867e9943bec89008ac3852cca,八恶人,0.696,0.304,0.086,0.326,0.283,0.131,0.067,0.107,0.332,0.668,0.138,0.491,0.14,0.231,0.41,0.13,0.46,0.32,0.11,0.27,0.18,0.05
1,c33b2c51d5cc8ebfce87dae8aac3f3f6,乌龙特工,0.708,0.292,0.106,0.33,0.268,0.117,0.075,0.104,0.267,0.733,0.141,0.472,0.169,0.218,0.38,0.11,0.51,0.32,0.11,0.27,0.18,0.05
2,c33ee9bd38bd25940cdf4b603b96a6d7,相爱相亲,0.393,0.607,0.059,0.304,0.29,0.151,0.093,0.102,0.631,0.369,0.167,0.412,0.194,0.227,0.49,0.18,0.33,0.46,0.12,0.19,0.07,0.06
3,c343fe7fd3530ef2cc3e8016046c7642,屏住呼吸,0.631,0.369,0.116,0.363,0.236,0.091,0.079,0.114,0.281,0.719,0.164,0.465,0.172,0.199,0.32,0.16,0.52,0.32,0.11,0.27,0.18,0.05
4,c3e8465a2641d7aa69aab8d45add78ba,不期而遇,0.393,0.607,0.105,0.391,0.26,0.11,0.063,0.07,0.685,0.315,0.134,0.401,0.202,0.263,0.31,0.2,0.49,0.44,0.14,0.21,0.07,0.05
